Adapted from radekosmulski [notebook](https://github.com/radekosmulski/whale/blob/master/first_submission.ipynb) for learning purpose on the playground competition instead of the original.

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 12.2MB/s 
     |████████████████████████████████| 1.1MB 59.2MB/s 
     |████████████████████████████████| 51kB 9.2MB/s 
     |████████████████████████████████| 194kB 66.8MB/s 
     |████████████████████████████████| 61kB 10.3MB/s 
Mounted at /content/gdrive


In [ ]:
root='/content/gdrive/MyDrive/'
base=f'{root}/Colab Notebooks/whale'

## Setup Kaggle and Data

In [ ]:
#Setup kaggle key
!mkdir -p ~/.kaggle
!cp '{base}/kaggle.json' ~/.kaggle/
!ls ~/.kaggle


kaggle.json


In [ ]:
#Download kaggle cli
! pip install --upgrade --force-reinstall --no-deps kaggle -q

     |████████████████████████████████| 61kB 8.4MB/s 


In [ ]:
from pathlib import Path

data_dir='data'
if not Path(data_dir).exists():
  ! mkdir -p {data_dir}
  
  #Download competition data
  ! kaggle competitions download -c whale-categorization-playground
  ! unzip -q whale-categorization-playground.zip

  ! mv train/train {data_dir}/
  ! mv test/test {data_dir}/
  ! mv train.csv {data_dir}/
  ! mv sample_submission.csv /

  !rm -f train
  !rm -f test

## Imports

In [ ]:
from nbdev.showdoc import *
from fastai.vision.all import *

import pandas as pd

## Setup

In [ ]:
SEED=42
set_seed(s=SEED, reproducible=True)

## Data

In [ ]:
df = pd.read_csv(f'{data_dir}/train.csv')
df.head()

,Image,Id
0,00022e1a.jpg,w_e15442c
1,000466c4.jpg,w_1287fbc
2,00087b01.jpg,w_da2efe0
3,001296d5.jpg,w_19e5482
4,0014cfdf.jpg,w_f22f3e3


In [ ]:
df.Id.value_counts()

new_whale    810
w_1287fbc     34
w_98baff9     27
w_7554f44     26
w_1eafe46     23
            ... 
w_c774326      1
w_740dfd4      1
w_65ec378      1
w_0c8a724      1
w_d26cc27      1
Name: Id, Length: 4251, dtype: int64

In [ ]:
(df.Id == 'new_whale').mean()

0.08223350253807106

In [ ]:
(df.Id.value_counts() == 1).mean()

0.5222300635144672

52% of all whales have only single image associated with them. 

8.2% of all images contain a new whale ie not a known whale.

[TODO] There is a superb writeup on what a solution to this problem might look like [here](https://www.kaggle.com/martinpiotte/whale-recognition-model-with-score-0-78563/notebook). In general, the conversation in the Kaggle [forum](https://www.kaggle.com/c/humpback-whale-identification/discussion) also seems to have some very informative threads.

In [ ]:
df.Id.nunique()

4251

In [ ]:
df.shape

(9850, 2)

## DataBlock

In [ ]:
SZ = 224
BS = 64
NUM_WORKERS = 12 
SEED = 42

In [ ]:
source=Path(data_dir)

In [ ]:
source

Path('data')

In [ ]:
data = DataBlock(
        blocks = (ImageBlock, CategoryBlock),
        get_x =ColReader(0, pref=source/'train'),
        get_y=ColReader(1),
        splitter=RandomSplitter(seed=SEED),
        item_tfms=Resize(SZ),
        batch_tfms=aug_transforms())

In [ ]:
#data.summary(source=df)

In [ ]:
dls = data.dataloaders(source=df, bs=BS, num_workers=NUM_WORKERS)

In [ ]:
#dls.show_batch()

In [ ]:
def map5(preds,targs):
    if type(preds) is list:
        return torch.cat([map5fast(p, targs, 5).view(1) for p in preds ]).mean()
    return map5kfast(preds,targs, 5)

## Modeling

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=[accuracy, map5])

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [ ]:
#learn.lr_find()

In [ ]:
#learn.fine_tune(5, 1e-2, freeze_epochs=1)